In [ ]:
import numpy as np
import pandas as pd 
import os
import gc
import sys
import time
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, StratifiedKFold
import xgboost as xgb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error

from transformers import AutoModel, AutoTokenizer
import json
from tensorflow.keras.models import load_model
import re
import pandas as pd
import string
import keras
from sklearn.svm import SVR
import math
import pickle
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM,Dropout,concatenate
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Activation, Embedding, LSTM,Dropout,Bidirectional,GRU
from keras.utils import plot_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Flatten ,Embedding,Input,Conv1D,GlobalAveragePooling1D,GlobalMaxPooling1D,Dropout,MaxPooling1D,Bidirectional,GRU,Concatenate
from keras.models import Sequential,Model
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
data_dir = '../input/commonlitreadabilityprize/'
train = pd.read_csv(data_dir + 'train.csv')
test = pd.read_csv(data_dir + 'test.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

target = train['target'].to_numpy()

In [ ]:
pickle_in = open("../input/elmo-embeddings-common-lit/Elmo Embeddings Common Lit/elmo_train_03032019.pickle", "rb")
train_embedding=pickle.load(pickle_in)

In [ ]:
pickle_in = open("../input/elmo-embeddings-common-lit/Elmo Embeddings Common Lit/elmo_test_03032019.pickle", "rb")
test_embedding=pickle.load(pickle_in)

In [ ]:
def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

In [ ]:
config = {
    'batch_size': 128,
    'max_len': 256,
    'seed': 42,
}

In [ ]:
mean_scores=[]
def get_preds(X,y,X_test,nfolds=20,C=10,kernel='rbf'):
    scores = list()
    preds = np.zeros((X_test.shape[0]))
    kf = KFold(n_splits=nfolds, shuffle=True, random_state=config['seed'])
    for k, (train_idx, valid_idx) in enumerate(kf.split(train)): 
        model = SVR(C=C,kernel=kernel,gamma='auto')
        train_x,train_y,val_x,val_y=X[train_idx], y[train_idx],X[valid_idx], y[valid_idx]
        
        
        model.fit(train_x,train_y)
        prediction = model.predict(val_x)
        score = rmse_score(prediction,val_y)
        print(f'Fold {k} , rmse score: {score}')
        scores.append(score)
        preds += model.predict(X_test)
        
    print("mean rmse",np.mean(scores))
    mean_scores.append(np.mean(scores))
    return np.array(preds)/nfolds

In [ ]:
preds1 = get_preds(train_embedding,target,test_embedding)

In [ ]:
submission = pd.DataFrame({'id':test.id,'target':preds1})
submission.to_csv('submission.csv',index=False)